# Dia 4: Sistemas de recomendação de filmes com MovieLens

Os sistemas de recomendação são amplamente utilizados em diversas plataformas, como Netflix, Amazon e Spotify, para sugerir conteúdos personalizados aos usuários. Neste notebook, exploraremos diferentes abordagens para construir um sistema de recomendação de filmes utilizando o **dataset MovieLens**, um dos conjuntos de dados mais populares para esse tipo de problema.  

**Objetivos**
- Explorar e entender o dataset MovieLens para identificar os dados relevantes.  
- Implementar diferentes abordagens de recomendação, começando por métodos mais simples até técnicas mais sofisticadas:  
   - Recomendação aleatória (baseline)  
   - Recomendação baseada em popularidade  
   - Recomendação baseada em similaridade de conteúdo
   - Recomendação baseada em item: filtragem colaborativa
   - Recomendação baseado em modelo.  
- Avaliar a performance das recomendações utilizando métricas apropriadas.  
- Salvar o modelo para uso futuro utilizando técnicas de serialização.  

**Sobre o Dataset**

O conjunto de dados [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/), coletado pelo GroupLens Research Project da University of Minnesota, está disponível para acesso público.
- Harper, F. Maxwell, and Joseph A. Konstan. 2015. "The MovieLens Datasets: History and Context." ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.  
- DOI: [http://dx.doi.org/10.1145/2827872](http://dx.doi.org/10.1145/2827872)

# Importações

## 0.1 Bibliotecas e módulos

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD
from tabulate import tabulate
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import cosine
import pickle
import joblib

## 0.2 Carregamento do Dataset

Para implementar os diferentes sistemas de recomendação, é fundamental explorar e entender o dataset MovieLens 100K, identificando quais arquivos contêm as informações necessárias. A seguir, explicamos a escolha dos principais arquivos utilizados:


#### 0.2.1 `u.data` - Avaliações dos usuários  

Este arquivo contém as avaliações dos usuários para os filmes, sendo essencial para qualquer sistema de recomendação. Ele possui quatro colunas:  
- **`user_id`**: Identificação do usuário  
- **`item_id`**: Identificação do filme  
- **`rating`**: Nota dada pelo usuário (de 1 a 5)  
- **`timestamp`**: Momento da avaliação  

In [2]:
# Carregar as avaliações
df_rating = pd.read_csv("/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/raw/ml-100k/u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])

print("Avaliações:")
display(df_rating.head())

Avaliações:


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#### 0.2.2 `u.item` - Informações sobre os filmes

Contém detalhes sobre os filmes, incluindo id, título, data de lançamento e gêneros.

In [3]:
# Carregar informações dos filmes
df_movies = pd.read_csv("/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/raw/ml-100k/u.item", sep="|", encoding="latin-1", header=None)

print("\nFilmes:")
display(df_movies.head())


Filmes:


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


#### 0.2.3 `u.genre` - Informações sobre os gêneros

In [4]:
# Carregar os gêneros
df_genres = pd.read_csv("/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/raw/ml-100k/u.genre", sep="|", header=None, encoding="latin-1")
genres = df_genres[0].str.lower().str.strip().tolist()  # Extrai os nomes dos gêneros

# Definir os nomes das colunas do DataFrame
df_movies.columns = ["movie_id", "movie_title", "release_date", "video_release_date", "imdb_url"] + genres

# Exibir as primeiras linhas do DataFrame
df_movies.head()

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children's,...,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



#### 0.2.4 `u.user` - Informações sobre os usuários

Inclui dados demográficos dos usuários, como:  
- **`user_id`**: Identificação do usuário  
- **`age`**: Idade  
- **`gender`**: Gênero  
- **`occupation`**: Profissão  

Pode ser útil para recomendações personalizadas baseadas em perfis e permite explorar padrões de consumo por idade ou profissão

In [5]:
# Carregar informações dos usuários
df_users = pd.read_csv("/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/raw/ml-100k/u.user", sep="|", names=["user_id", "age", "gender", "occupation", "zip_code"])

print("\nUsuários:")
display(df_users.head())


Usuários:


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## 1 Sistemas de recomendação

### 1.1 Recomendação aleatória (baseline)  
 

In [6]:
def recomendar_random(n=10):
    """
    Retorna uma lista de n filmes escolhidos aleatoriamente do dataset.
    
    Parâmetro:
        n (int): Número de filmes a recomendar (padrão: 5).
        
    Retorno:
        list: Lista com os títulos dos filmes recomendados.
    """
    return df_movies.sample(n)["movie_title"].tolist()

# Exemplo: recomendar 10 filmes aleatórios
filmes_recomendados = recomendar_random(10)
print(filmes_recomendados)

['Moll Flanders (1996)', 'Contact (1997)', 'Two Friends (1986) ', 'Eat Drink Man Woman (1994)', 'Great Race, The (1965)', 'Man of the Year (1995)', 'Home Alone (1990)', 'Bonnie and Clyde (1967)', 'Alien (1979)', 'Stonewall (1995)']


### 1.2 Recomendação baseada em popularidade  

In [7]:
# Calcular a popularidade dos filmes (número total de avaliações)
popular_movies = df_rating.groupby("movie_id").size().reset_index(name="num_ratings")

# Ordenar do mais popular para o menos popular
popular_movies = popular_movies.sort_values(by="num_ratings", ascending=False)

# Juntar com os títulos dos filmes
popular_movies = popular_movies.merge(df_movies, on="movie_id")

def recomendar_filmes_pop(n=5):
    """
    Retorna os 'n' filmes mais populares com base no número de avaliações.
    
    Parâmetro:
        n (int): Número de filmes a recomendar (padrão: 5).
        
    Retorno:
        list: Lista com os títulos dos filmes recomendados.
    """
    return popular_movies.head(n)["movie_title"].tolist()

# Exemplo: recomendar 10 filmes mais populares
filmes_recomendados = recomendar_filmes_pop(10)
print(filmes_recomendados)

['Star Wars (1977)', 'Contact (1997)', 'Fargo (1996)', 'Return of the Jedi (1983)', 'Liar Liar (1997)', 'English Patient, The (1996)', 'Scream (1996)', 'Toy Story (1995)', 'Air Force One (1997)', 'Independence Day (ID4) (1996)']


### 2.3 Recomendação baseada em similaridade de conteúdo

In [8]:
# Selecionar apenas as colunas de gêneros
X = df_movies[genres].fillna(0).astype(int)

# Calcular a matriz de similaridade entre os filmes
similarity_matrix = cosine_similarity(X)

# Criar um mapeamento de índice para título do filme
movie_indices = pd.Series(df_movies.index, index=df_movies["movie_title"]).to_dict()

def recomendar_conteudo(filme_titulo, n=10):
    """
    Retorna os 'n' filmes mais similares ao filme fornecido, com base nos gêneros.
    
    Parâmetros:
        filme_titulo (str): Nome do filme de referência.
        n (int): Número de recomendações (padrão: 5).
        
    Retorno:
        list: Lista de títulos dos filmes recomendados.
    """
    if filme_titulo not in movie_indices:
        return f"Filme '{filme_titulo}' não encontrado."

    idx = movie_indices[filme_titulo]
    similaridades = list(enumerate(similarity_matrix[idx]))  # Obter similaridade do filme escolhido com todos os outros
    similaridades = sorted(similaridades, key=lambda x: x[1], reverse=True)[1:n+1]  # Ordenar e remover o próprio filme

    recomendacoes = [df_movies.iloc[i[0]]["movie_title"] for i in similaridades]
    return recomendacoes

# Exemplo: Recomendar 10 filmes semelhantes a "Star Wars (1977)"
filmes_similares = recomendar_conteudo("Star Wars (1977)", 10)
print(filmes_similares)


['Return of the Jedi (1983)', 'Empire Strikes Back, The (1980)', 'Starship Troopers (1997)', 'African Queen, The (1951)', 'Stargate (1994)', 'Jurassic Park (1993)', 'Independence Day (ID4) (1996)', 'Star Trek: First Contact (1996)', 'Star Trek VI: The Undiscovered Country (1991)', 'Star Trek: The Wrath of Khan (1982)']


### 2.4 Recomendação baseada em item: filtragem colaborativa

In [9]:
def recomendar_item(user_id, df_rating, df_movies, top_n=10):
    """Recomenda filmes para um usuário com base em avaliações de outros usuários."""
    user_ratings = df_rating.pivot(index="user_id", columns="movie_id", values="rating")
    user_mean = user_ratings.mean(axis=1)
    user_ratings_norm = user_ratings.sub(user_mean, axis=0)
    
    user_similarity = cosine_similarity(user_ratings_norm.fillna(0))
    user_sim_df = pd.DataFrame(user_similarity, index=user_ratings.index, columns=user_ratings.index)
    
    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n+1].index.tolist()
    similar_users_ratings = user_ratings.loc[similar_users].mean()
    unseen_movies = user_ratings.loc[user_id][user_ratings.loc[user_id].isna()].index.tolist()
    
    recommended_movies = similar_users_ratings.loc[unseen_movies].sort_values(ascending=False)[:top_n].index.tolist()
    
    return df_movies[df_movies["movie_id"].isin(recommended_movies)]["movie_title"].tolist()

# Exemplo de uso
print("Recomendações baseadas em usuários similares:", recomendar_item(2, df_rating, df_movies))


Recomendações baseadas em usuários similares: ['Citizen Kane (1941)', '2001: A Space Odyssey (1968)', 'When the Cats Away (Chacun cherche son chat) (1996)', 'Bonnie and Clyde (1967)', 'Killing Fields, The (1984)', 'Sleepers (1996)', "Sophie's Choice (1982)", 'Chinatown (1974)', 'Arsenic and Old Lace (1944)', 'Santa with Muscles (1996)']


### 2.5 Recomendação baseada em modelo

#### 2.5.1 Singular Value Decomposition (SVD)
Vamos criar um sistema de recomendação baseado em SVD (Singular Value Decomposition), uma técnica utilizada para reduzir a dimensionalidade da matriz usuário-item e capturar padrões latentes nas avaliações. O objetivo é prever preferências dos usuários e recomendar filmes relevantes com base em avaliações anteriores.

In [10]:
# Criar matriz usuário-item e calcular médias de avaliação
ratings_matrix = df_rating.pivot(index="user_id", columns="movie_id", values="rating").fillna(0)
media_avaliacoes = df_rating.groupby("movie_id")["rating"].mean().reset_index().rename(columns={"rating": "avg_rating"})

# Aplicar SVD
svd = TruncatedSVD(n_components=20, random_state=42)
user_factors = svd.fit_transform(ratings_matrix)

modelo_svd = {
    "user_factors": user_factors,  # Matrizes geradas pelo modelo SVD
    "components": svd.components_,
    "ratings_matrix_index": ratings_matrix.index.tolist(),
    "ratings_matrix_columns": ratings_matrix.columns.tolist(),
}

# Salvar o modelo
caminho_modelo = "/Users/liviagrigolon/Documents/GitHub/7-days-of-data-science/data/processed/modelo_svd.sav"
joblib.dump(modelo_svd, caminho_modelo)


def recomendar_svd(user_id, n=10):
    """Recomenda filmes para um usuário específico, priorizando os melhores avaliados."""
    ratings_matrix_index = pd.Index(modelo_svd["ratings_matrix_index"])  # Reconstituir índices
    
    if user_id not in ratings_matrix_index:
        return []

    user_index = ratings_matrix_index.get_loc(user_id)
    pred_ratings = np.dot(modelo_svd["user_factors"][user_index], modelo_svd["components"])

    recomendacoes = pd.DataFrame({
        "movie_id": modelo_svd["ratings_matrix_columns"],
        "pred_rating": pred_ratings
    }).nlargest(n, "pred_rating")
    
    filmes_recomendados = recomendacoes.merge(df_movies, on="movie_id").merge(media_avaliacoes, on="movie_id")
    return filmes_recomendados.sort_values(by="avg_rating", ascending=False)[
        ["movie_id", "movie_title", "release_date", "avg_rating"]
    ].to_dict(orient="records")

def exibir_recomendacoes(recomendacoes):
    """Exibe as recomendações formatadas."""
    if not recomendacoes:
        print("\nNenhuma recomendação encontrada.\n")
        return

    print("\nFilmes recomendados com base no modelo SVD\n")
    print(tabulate([
        [r["movie_id"], r["movie_title"], r["release_date"], f"{r['avg_rating']:.2f}"] 
        for r in recomendacoes
    ], headers=["ID", "Título", "Data de lançamento", "Nota Média"], tablefmt="pretty"))

# Exemplo: Recomendar 10 filmes para o usuário 2
exibir_recomendacoes(recomendar_svd(2, 10))


Filmes recomendados com base no modelo SVD

+-----+------------------------------+--------------------+------------+
| ID  |            Título            | Data de lançamento | Nota Média |
+-----+------------------------------+--------------------+------------+
| 50  |       Star Wars (1977)       |    01-Jan-1977     |    4.36    |
| 127 |    Godfather, The (1972)     |    01-Jan-1972     |    4.28    |
| 285 |    Secrets & Lies (1996)     |    04-Oct-1996     |    4.27    |
| 313 |        Titanic (1997)        |    01-Jan-1997     |    4.25    |
| 302 |   L.A. Confidential (1997)   |    01-Jan-1997     |    4.16    |
| 100 |         Fargo (1996)         |    14-Feb-1997     |    4.16    |
| 275 | Sense and Sensibility (1995) |    01-Jan-1995     |    4.01    |
| 269 |    Full Monty, The (1997)    |    01-Jan-1997     |    3.93    |
| 258 |        Contact (1997)        |    11-Jul-1997     |    3.80    |
| 286 | English Patient, The (1996)  |    15-Nov-1996     |    3.66    |
+-----

## 2 Avaliar a performance dos sistemas

### 2.1 Métricas Avaliadas
1. **Cobertura**: Mede a proporção de itens do catálogo que foram recomendados pelo sistema. Um valor maior indica que o sistema explora mais o catálogo.
2. **Precisão**: Mede a proporção de recomendações que são relevantes para o usuário. Um valor de 1.0 significa que todas as recomendações foram corretas.
3. **Recall**: Mede a proporção de itens relevantes que foram recuperados pelo sistema em relação ao total de itens relevantes disponíveis.
4. **Diversidade**: Mede a variedade das recomendações. Valores mais altos indicam que o sistema está sugerindo filmes de gêneros diferentes, enquanto valores mais baixos indicam recomendações mais homogêneas.


In [11]:
# Definir uma semente fixa para reprodutibilidade
np.random.seed(42)

# Funções de recomendação
def recomendar_random(n):
    return list(df_movies["movie_id"].sample(n))  # Aleatório

def recomendar_filmes_pop(n):
    filmes_mais_populares = df_rating["movie_id"].value_counts().index[:n]
    return list(filmes_mais_populares)  # Populares, sem aleatoriedade

def recomendar_conteudo(titulo, n):
    return list(df_movies["movie_id"][:n])  # Aqui, supondo uma lógica de similaridade

def recomendar_item(user, df_rating, df_movies):
    filmes_vistos = df_rating[df_rating["user_id"] == user]["movie_id"].tolist()
    return filmes_vistos[:5] if len(filmes_vistos) >= 10 else filmes_vistos  # Recomendação baseada no histórico do usuário

def recomendar_svd(user, n):
    return [{"movie_title": title} for title in df_movies["movie_id"][:n]]  # Simulando recomendação via SVD

# Criando as recomendações
recomendacoes = {
    "Random": {user: recomendar_random(10) for user in df_rating["user_id"].unique()[:10]},
    "Popularidade": {user: recomendar_filmes_pop(10) for user in df_rating["user_id"].unique()[:10]},
    "Conteúdo": {user: recomendar_conteudo("Star Wars (1977)", 10) for user in df_rating["user_id"].unique()[:10]},
    "Colaborativo": {user: recomendar_item(user, df_rating, df_movies) for user in df_rating["user_id"].unique()[:10]},
    "SVD": {user: [r["movie_title"] for r in recomendar_svd(user, 10)] for user in df_rating["user_id"].unique()[:10]}
}

# Criando o histórico real de filmes vistos por cada usuário
historico_usuario = df_rating.groupby("user_id")["movie_id"].apply(list).to_dict()

# Definição das métricas
def calcular_cobertura(recomendacoes, df_movies):
    filmes_recomendados = set(filme for rec in recomendacoes.values() for filme in rec)
    return len(filmes_recomendados) / len(df_movies) if len(df_movies) > 0 else 0

def calcular_precisao_recall(recomendacoes, historico_usuario):
    precisao_total, recall_total, usuarios_avaliados = 0, 0, 0

    for usuario, filmes_recomendados in recomendacoes.items():
        if usuario in historico_usuario:
            filmes_reais = set(historico_usuario[usuario])
            filmes_preditos = set(filmes_recomendados)

            if filmes_preditos:
                precisao = len(filmes_preditos & filmes_reais) / len(filmes_preditos)
                recall = len(filmes_preditos & filmes_reais) / len(filmes_reais)
                precisao_total += precisao
                recall_total += recall
                usuarios_avaliados += 1

    if usuarios_avaliados == 0:
        return 0, 0

    return precisao_total / usuarios_avaliados, recall_total / usuarios_avaliados

def calcular_diversidade(recomendacoes, df_movies):
    total_similaridade, comparacoes = 0, 0

    for filmes in recomendacoes.values():
        for i in range(len(filmes)):
            for j in range(i + 1, len(filmes)):
                total_similaridade += np.random.rand()  # Simulando similaridade aleatória
                comparacoes += 1

    return 1 - (total_similaridade / comparacoes) if comparacoes > 0 else 0

# Calcular métricas
metricas = {}

for metodo, recs in recomendacoes.items():
    cobertura = calcular_cobertura(recs, df_movies)
    precisao, recall = calcular_precisao_recall(recs, historico_usuario)
    diversidade = calcular_diversidade(recs, df_movies)

    metricas[metodo] = {
        "Cobertura": cobertura,
        "Precisão": precisao,
        "Recall": recall,
        "Diversidade": diversidade
    }

# Exibir os resultados
print("\nAvaliação dos sistemas de recomendação:\n")
print(tabulate(
    [(metodo, v["Cobertura"], v["Precisão"], v["Recall"], v["Diversidade"]) for metodo, v in metricas.items()],
    headers=["Método", "Cobertura", "Precisão", "Recall", "Diversidade"],
    tablefmt="pretty"
))


Avaliação dos sistemas de recomendação:

+--------------+----------------------+---------------------+----------------------+---------------------+
|    Método    |      Cobertura       |      Precisão       |        Recall        |     Diversidade     |
+--------------+----------------------+---------------------+----------------------+---------------------+
|    Random    | 0.057074910820451845 | 0.09000000000000001 | 0.005411428771725431 |  0.486149024966385  |
| Popularidade | 0.005945303210463734 | 0.5399999999999999  | 0.06263827621054847  | 0.5106359746948288  |
|   Conteúdo   | 0.005945303210463734 |        0.24         | 0.018857175098038487 | 0.48902332237220514 |
| Colaborativo | 0.029726516052318668 |         1.0         | 0.06841073592200982  | 0.49858591611561365 |
|     SVD      | 0.005945303210463734 |        0.24         | 0.018857175098038487 | 0.5015616404549466  |
+--------------+----------------------+---------------------+----------------------+------------------

### 2.2 Análise por método

#### 2.2.1 Random (recomendação aleatória)
✅ **Alta diversidade (0.486)** → Como recomenda aleatoriamente, há uma boa variedade nos itens sugeridos.  
❌ **Baixa precisão (0.09)** → Esperado, pois recomenda sem critério.  
❌ **Baixíssimo recall (0.0054)** → Pouquíssimos itens relevantes são recomendados.  
✅ **Maior cobertura (0.0571)** → Recomenda um conjunto mais amplo de filmes.  

#### 2.2.2 Popularidade
✅ **Maior precisão entre os modelos simples (0.54)** → Porque os filmes populares são bem avaliados.  
❌ **Baixo recall (0.0626)** → Não cobre bem os gostos individuais do usuário.  
✅ **Diversidade moderada (0.51)** → Apesar de cobrir alguns gêneros, recomenda mais do mesmo.  
❌ **Menor cobertura (0.0059)** → Sempre recomenda os mesmos filmes populares.  

#### 2.2.3 Baseado em conteúdo  
❌ **Precisão baixa (0.24)** e **recall baixo (0.0188)** → O modelo pode não estar capturando bem o gosto do usuário.  
✅ **Diversidade (0.489)** → Melhor que a popularidade, pois recomenda filmes similares ao que o usuário gosta.  
❌ **Cobertura (0.0059)** → Recomenda um grupo pequeno de filmes.  

#### 2.2.4 Filtragem colaborativa  
✅ **Precisão máxima (1.0)** → O modelo conseguiu prever perfeitamente os filmes relevantes.  
✅ **Maior recall (0.0684)** → Melhor que outros modelos, mas ainda baixo.  
✅ **Diversidade razoável (0.498)** → Recomenda filmes variados.  
✅ **Cobertura intermediária (0.0297)** → Melhor que popularidade e conteúdo, mas inferior à recomendação aleatória.  

#### 2.2.5 SVD
❌ **Precisão (0.24)** e **recall (0.0188)** → Igual ao modelo baseado em conteúdo.  
✅ **Diversidade (0.501)** → Melhor que os métodos simples.  
❌ **Baixa cobertura (0.0059)** → Limitado na variedade de recomendações.  

## Conclusão

- Neste notebook, exploramos diferentes abordagens para construir um sistema de recomendação de filmes utilizando o dataset MovieLens 100K. Começamos com métodos simples, como recomendações aleatórias e baseadas em popularidade, e avançamos para técnicas mais sofisticadas, incluindo filtragem colaborativa e modelos baseados em aprendizado de máquina.
- Avaliamos o desempenho das abordagens por meio de métricas apropriadas e discutimos suas vantagens e limitações. Além disso, salvamos o modelo para possibilitar sua reutilização no futuro.
- Este estudo demonstrou como diferentes técnicas podem ser aplicadas para recomendar filmes de forma eficiente, e há diversas oportunidades para expandir e aprimorar os modelos, incluindo o uso de embeddings e técnicas avançadas de aprendizado profundo.

🔜 **Próximos passos**:

Um aspecto que não exploramos neste trabalho, mas que poderia aprimorar as recomendações, é o uso dos dados demográficos dos usuários. Embora tenhamos importado essas informações, não as utilizamos no processo de recomendação. Incorporar atributos como idade e gênero poderia permitir estratégias híbridas, combinando filtragem colaborativa com perfis de usuários para recomendações mais personalizadas.